In [1]:
from scipy.io import loadmat
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.losses import Loss
from calibration_stats import ExpectedCalibrationError
import time 
from matplotlib import pyplot as plt

import sys
sys.path.insert(1, '/home/thlarsen/ood_detection')

from helper import load_dataset_sev, load_dataset_c, load_mnist_model, rgb_img_to_vec



In [2]:
train_raw = loadmat('/home/thlarsen/ood_detection/learn_uncertainty/datasets/svhndataset/train_32x32.mat')
test_raw = loadmat('/home/thlarsen/ood_detection/learn_uncertainty/datasets/svhndataset/test_32x32.mat')


N=200

x_train_raw = np.array(train_raw['X'])
x_test_raw = np.array(test_raw['X'])

x_train_raw = np.moveaxis(x_train_raw, -1, 0)
x_test_raw = np.moveaxis(x_test_raw, -1, 0)

y_train = train_raw['y']
y_test = test_raw['y']


(26032, 32, 32, 3)
(26032, 1)
(73257, 32, 32, 3)


In [22]:
(_, _), (x_test_m, y_test_m) = keras.datasets.mnist.load_data()

# x_train = np.pad(x_train, ((0,0),(2,2),(2,2)), 'constant')
x_test_m = np.pad(x_test_m, ((0,0),(2,2),(2,2)), 'constant')

# print(x_train.shape)

# x_train = x_train.reshape((-1, 32, 32, 1)).repeat(3, axis=3)
x_test_m = x_test_m.reshape((-1, 32, 32, 1)).repeat(3, axis=3).reshape(-1, 3072)
print(x_test_m.shape)

# print(x_train.shape)

# x_train = bnn_cast(x_train)


# Prepare the training dataset.
test_mnist = tf.data.Dataset.from_tensor_slices((x_test_m, y_test_m))
test_mnist = test_mnist.shuffle(buffer_size=1024).batch(batch_size=64)

(10000, 3072)


In [8]:
x_train = x_train_raw.reshape(-1, 3072)
x_test = x_test_raw.reshape(-1, 3072)


test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.shuffle(buffer_size=1024).batch(batch_size=64)


(73257, 3072)


In [16]:
model1 = load_mnist_model(lr=1e-4, w=.1,model_arch='seq', train_alg='ece') 
model2 = load_mnist_model(lr=1e-4, w=.1,model_arch='seq', train_alg='ece_shift') 
model3 = load_mnist_model(lr=1e-4, w=0,model_arch='seq', train_alg='ece') 


In [23]:
for model in [model1, model2, model3]:
    acc_fn = keras.metrics.SparseCategoricalAccuracy()

    for step, (x_batch, y_batch) in enumerate(test_mnist):
        y_preds = model.predict(x_batch)
        acc_fn.update_state(y_batch, y_preds)
    print(acc_fn.result())

tf.Tensor(0.9426, shape=(), dtype=float32)
tf.Tensor(0.9615, shape=(), dtype=float32)
tf.Tensor(0.9522, shape=(), dtype=float32)
